In [1]:
# Pandas and numpy to work with dataframes
import pandas as pd 
import numpy as np
from datetime import datetime
# Requests to use Runescape wiki api (OLD SCHOOL, NOT THE NEW ONE)
import requests 
# We want to include lists of items such as food, which are not standardized (very different names!)
from selenium import webdriver
from selenium.webdriver.common.by import By

In [2]:
# Initiate driver
driver = webdriver.Chrome('chromedriver')
# Retrieve the webpage
driver.get('https://oldschool.runescape.wiki/w/Food')
# We are interested in the names found in the popular foods section:
# class = "wikitable sortable align-center-1 align-right-6 align-right-7 jquery-tablesorter"


In [3]:
# We need to do mapping ~ get all the item ids from here.
headers = {'User-agent': 'Time_Series_Prices - @Mr. Yellow#3368'}
response_mapping = requests.get('https://prices.runescape.wiki/api/v1/osrs/mapping', headers = headers)
# Set up contain function:
def contains(a: str, b: str):
    """

    :param a:
    :param b:
    :return:
    """
    a = a.lower()
    b = b.lower()
    return(a in b)
# response_mapping.text
# Retrieve lobster item code:
list1 = []
list2 = []
# Lists for id and name
id_list = []
name_list = []
# Produce a list of names, and their associated ID, we will need this to conduct the requests.
for item in response_mapping.json():
    if contains('rune', item['name']):
        list1.append(item['name'])
        list2.append(item['id'])
# If we split it, second one should be rune...
index_list = 0 
while index_list in range(len(list1)):
    if list1[index_list].split(' ')[1] == 'rune' and len(list1[index_list].split(' ')) == 2:
        name_list.append(list1[index_list])
        id_list.append(list2[index_list])
    else:
        pass
    index_list += 1

print(name_list)
print(id_list)

['Air rune', 'Astral rune', 'Blood rune', 'Body rune', 'Chaos rune', 'Cosmic rune', 'Death rune', 'Dust rune', 'Earth rune', 'Fire rune', 'Lava rune', 'Law rune', 'Mind rune', 'Mist rune', 'Mud rune', 'Nature rune', 'Smoke rune', 'Soul rune', 'Steam rune', 'Water rune', 'Wrath rune']
[556, 9075, 565, 559, 562, 564, 560, 4696, 557, 554, 4699, 563, 558, 4695, 4698, 561, 4697, 566, 4694, 555, 21880]


In [4]:
# Gather a list of the responses
resp_list = []
for id_items in id_list:
    response_price_history = requests.get('https://prices.runescape.wiki/api/v1/osrs/timeseries?timestep=5m&id={}'.format(id_items)
                                          , headers = headers)
    resp_list.append(response_price_history.json()['data'])
print(resp_list)

[[{'timestamp': 1677256200, 'avgHighPrice': 5, 'avgLowPrice': 4, 'highPriceVolume': 144244, 'lowPriceVolume': 8363}, {'timestamp': 1677256500, 'avgHighPrice': 5, 'avgLowPrice': 4, 'highPriceVolume': 192627, 'lowPriceVolume': 2793}, {'timestamp': 1677256800, 'avgHighPrice': 5, 'avgLowPrice': 4, 'highPriceVolume': 122781, 'lowPriceVolume': 150}, {'timestamp': 1677257100, 'avgHighPrice': 5, 'avgLowPrice': 4, 'highPriceVolume': 165300, 'lowPriceVolume': 140260}, {'timestamp': 1677257400, 'avgHighPrice': 5, 'avgLowPrice': 4, 'highPriceVolume': 460614, 'lowPriceVolume': 102}, {'timestamp': 1677257700, 'avgHighPrice': 5, 'avgLowPrice': 4, 'highPriceVolume': 223439, 'lowPriceVolume': 160}, {'timestamp': 1677258000, 'avgHighPrice': 4, 'avgLowPrice': 4, 'highPriceVolume': 151145, 'lowPriceVolume': 12725}, {'timestamp': 1677258300, 'avgHighPrice': 5, 'avgLowPrice': None, 'highPriceVolume': 45337, 'lowPriceVolume': 0}, {'timestamp': 1677258600, 'avgHighPrice': 4, 'avgLowPrice': 4, 'highPriceVolume

In [5]:
# we wish to ignore the cases where avglowprice or avghighprice or rather change calculations...
# EDIT FILE TO DAY2
full_name_list = []
dt_list = []
price_list = []
total_trade_volume = []
high_trade_volume = []
low_trade_volume = []
high_price = []
low_price = []
ind1 = 0
while ind1 in range(len(resp_list)):
    for item in resp_list[ind1]:
        # Can do name and dt_list right here right now
        full_name_list.append(name_list[ind1])
        total_trade_volume.append((item['highPriceVolume'] + item['lowPriceVolume']))
        high_trade_volume.append(item['highPriceVolume'])
        low_trade_volume.append(item['lowPriceVolume'])
        high_price.append(item['avgHighPrice'])
        low_price.append(item['avgLowPrice'])
        dt_list.append(datetime.utcfromtimestamp(int(item['timestamp'])).strftime('%Y-%m-%d %H:%M:%S'))
        if item['avgHighPrice'] == None:
            price_list.append(item['avgLowPrice'])
        elif item['avgLowPrice'] == None:
            price_list.append(item['avgHighPrice'])
        elif item['avgLowPrice'] == None and item['acgHighPrice'] == None:
            price_list.append(np.nan)
        else:
            high_share = item['highPriceVolume'] / (item['highPriceVolume'] + item['lowPriceVolume'])
            low_share = item['lowPriceVolume'] / (item['highPriceVolume'] + item['lowPriceVolume'])
            avg_price = item['avgHighPrice']*high_share + item['avgLowPrice']*low_share
            price_list.append(avg_price)
    ind1 += 1
rune_price_df = pd.DataFrame(list(zip(full_name_list, dt_list, price_list, total_trade_volume,
                                     high_trade_volume, low_trade_volume, high_price, low_price)), 
                             columns = ['Item Name', 'Date', 'Average Price', 'Total Trade Volume', 'High Price Trade Volume',
                                        'Low Price Trade Volume', 'Average High Price', 'Average Low Price'])
rune_price_df.to_csv(r'C:\Users\huang\OneDrive\Documents\ECO481_Assign\rune_price_day1_v2.csv')
